In [2]:
from LSTM_for_rainfall_runoff_modelling import CamelsTXT

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import tqdm
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


from LSTM_for_rainfall_runoff_modelling import CamelsTXT, Model, DataLoader, train_epoch, eval_model, calc_nse

def setup_data(basin, sequence_length):
    # Training data
    start_date_train = pd.to_datetime("1980-10-01", format="%Y-%m-%d")
    end_date_train = pd.to_datetime("1995-09-30", format="%Y-%m-%d")
    ds_train = CamelsTXT(basin, seq_length=sequence_length, period="train", dates=[start_date_train, end_date_train])
    tr_loader = DataLoader(ds_train, batch_size=256, shuffle=True)

    # Validation data
    means = ds_train.get_means()
    stds = ds_train.get_stds()
    start_date_val = pd.to_datetime("1995-10-01", format="%Y-%m-%d")
    end_date_val = pd.to_datetime("2000-09-30", format="%Y-%m-%d")
    ds_val = CamelsTXT(basin, seq_length=sequence_length, period="eval", dates=[start_date_val, end_date_val],
                         means=means, stds=stds)
    val_loader = DataLoader(ds_val, batch_size=2048, shuffle=False)

    # Test data
    start_date_test = pd.to_datetime("2011-01-01", format="%Y-%m-%d")
    end_date_test = pd.to_datetime("2011-01-01", format="%Y-%m-%d")
    ds_test = CamelsTXT(basin, period="eval", dates=[start_date_test, end_date_test],
                         means=means, stds=stds)
    test_loader = DataLoader(ds_test, batch_size=2048, shuffle=False)

    return tr_loader, val_loader, test_loader, ds_val, ds_test

def setup_model(hidden_size, dropout_rate, learning_rate):
    model = Model(hidden_size=hidden_size, dropout_rate=dropout_rate).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    loss_func = nn.MSELoss()
    return model, optimizer, loss_func

def train_and_eval_model(model, optimizer, tr_loader, val_loader, n_epochs):
    for i in range(n_epochs):
        train_epoch(model, optimizer, tr_loader, loss_func, i+1)
        obs, preds = eval_model(model, val_loader)
        preds = ds_val.local_rescale(preds.numpy(), variable='output')
        nse = calc_nse(obs.numpy(), preds)
        tqdm.tqdm.write(f"Validation NSE: {nse:.2f}")

def evaluate_and_plot_results(model, test_loader, ds_val, ds_test):
    start_date_test = pd.to_datetime("2000-01-01", format="%Y-%m-%d")
    end_date_test = pd.to_datetime("2011-01-30", format="%Y-%m-%d")
    obs, preds = eval_model(model, test_loader)
    preds = ds_val.local_rescale(preds.numpy(), variable='output')
    obs = obs.numpy()
    nse = calc_nse(obs, preds)
    # Plot results
    start_date = end_date_test
    end_date = end_date_test + pd.DateOffset(days=1)
    date_range = pd.date_range(start_date, end_date, freq='D').date
    print(obs.shape)
    print(preds.shape)
    # Aggregate the observed and predicted discharge values for each day
    obs_daily_mean = obs.mean(axis=1)
    preds_daily_mean = preds.mean(axis=1)
    
    # Plot results
    print(date_range)
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(date_range, obs_daily_mean, label="observation")
    ax.plot(date_range, preds_daily_mean, label="prediction")
    ax.legend()
    ax.set_title(f"Basin {basin} - Test set NSE: {nse:.3f}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Discharge (mm/d)")
    ax.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))  # Set the x-axis tick locator to use integer values
    ax.xaxis.set_tick_params(rotation=90)  # Rotate x-axis labels for better readability

C:\Users\Homayoun\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Homayoun\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\Homayoun\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
    basin = '01022500'
    hidden_size = 10
    dropout_rate = 0.0
    learning_rate = 1e-3
    sequence_length = 365

    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    tr_loader, val_loader, test_loader, ds_val, ds_test = setup_data(basin, sequence_length)
    model, optimizer, loss_func = setup_model(hidden_size, dropout_rate, learning_rate)

    n_epochs = 1
    train_and_eval_model(model, optimizer, tr_loader, val_loader, n_epochs)
    evaluate_and_plot_results(model, test_loader, ds_val, ds_test)

RuntimeError: No forcing file file found for Basin 01022500